In [2]:
import numpy as np
import pandas as pd
import os

In [6]:
input_path = './training_data/'
output_path = './result_data/'

In [7]:
file_list = os.listdir(input_path)

In [8]:
file_list.sort()
file_list

['WA_TR_20191015.xlsx',
 'WA_TR_20191016.xlsx',
 'WA_TR_20191018.xlsx',
 'WA_TR_20191019.xlsx',
 'WA_TR_20191020.xlsx',
 'WA_TR_20191022.xlsx',
 'WA_TR_20191023.xlsx',
 'WA_TR_20191024.xlsx',
 'WA_TR_20191025.xlsx']

In [9]:
result_colmns = ['Name', 'Position', 'Date', 'Type_1', 'Type_2', 'Type_3', 'Height', 'Weight', 'RPE', 'TR_time_(min)', 'Total_dist_(m)', 'Dist_per_min_(m)', 'HSR+Sprint_(m)', 'MSR_(m)', 'HSR_(m)', 'Sprint_(m)', 'Accel_(m)', 'Decel_(m)', 'Max_speed_(km/h)', 'Load', 'Mono', 'Strain', 'EWAM']

In [10]:
def trainig_merge(file_name):
    file_path = input_path+file_name
    xl = pd.ExcelFile(file_path)
    
    sheet_num = len(xl.sheet_names)
    print('{} has {} sheets'.format(file_name, sheet_num))
    
    train_df = None
    
    if sheet_num==0:
        df = xl.parse(xl.sheet_names)
        df['Load'] = df['RPE']*df['TR_time_(min)']
        train_df = df        
    else:
        df_list = []
        for i, sn in enumerate(xl.sheet_names):
            print("Now try to merge Training Dataset : {} at {}".format(sn, file_path))
            df = xl.parse(sn)
            df['Load'] = df['RPE']*df['TR_time_(min)']
            if i == 0:
                train_df = df
            else:
                df_list.append(df)

        for df in df_list:
            train_df['Type'] = train_df['Type']+ ' / ' +df['Type']
    #         trian_df['Height'] = df['Height']
    #         trian_df['Weight'] = df['Weight']
            train_df['RPE'] = train_df['RPE'] + df['RPE'] 
            train_df['TR_time_(min)'] = train_df['TR_time_(min)'] + df['TR_time_(min)']
            train_df['Total_dist_(m)'] =  train_df['Total_dist_(m)'] + df['Total_dist_(m)']
            train_df['Dist_per_min_(m)'] = train_df['Dist_per_min_(m)'] + df['Dist_per_min_(m)']
            train_df['HSR+Sprint_(m)'] = train_df['HSR+Sprint_(m)'] + df['HSR+Sprint_(m)'] 
            train_df['MSR_(m)'] = train_df['MSR_(m)'] + df['MSR_(m)']
            train_df['HSR_(m)'] = train_df['HSR_(m)'] + df['HSR_(m)']
            train_df['Sprint_(m)'] = train_df['Sprint_(m)'] + df['Sprint_(m)']
            train_df['Accel_(m)'] = train_df['Accel_(m)'] + df['Accel_(m)']
            train_df['Decel_(m)'] = train_df['Decel_(m)'] + df['Decel_(m)']
            train_df['Max_speed_(km/h)'] = np.where((train_df['Max_speed_(km/h)'] >= df['Max_speed_(km/h)']), train_df['Max_speed_(km/h)'], df['Max_speed_(km/h)'])
            train_df['Load'] = train_df['Load'] + df['Load'] 

        # Average colums
        train_df['RPE'] = train_df['RPE'] / sheet_num
        train_df['Dist_per_min_(m)'] = train_df['Dist_per_min_(m)'] / sheet_num
    
    print('All Training Dataets were successfully merged.')
    return train_df

In [15]:
def result_by_player(train_df):


    info_df = train_df.loc[0:0, 'Date':'Type']
    mean_df = train_df.loc[:,'Height':].mean().to_frame().T
    mean_df = info_df.join(mean_df)


    if os.path.isfile('./result_data/result_by_player.xlsx'):
        writer = pd.ExcelWriter('./result_data/result_by_player.xlsx')
        team_df = pd.read_excel('./result_data/result_by_player.xlsx', sheet_name='Team sheet')

        temp_df = team_df.iloc[:,:-3]
        temp_df = temp_df.append(mean_df, ignore_index=True)


        t_day = len(temp_df)

        if t_day <= 7:
            mean_df['Mono'] = temp_df['Load'].mean()/temp_df['Load'].std()
            mean_df['Strain'] = temp_df['Load'].sum()*mean_df['Mono']
            mean_df['EWAM'] = temp_df['Load'].sum()/temp_df['Load'].sum()
        elif t_day <=28:
            week_n = t_day - 7
            mean_df['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
            mean_df['Strain'] = temp_df.loc[week_n:,'Load'].sum()*mean_df['Mono']
            mean_df['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[week_n:,'Load'].sum()
        else:
            week_n = t_day - 7
            month_n = t_day - 28
            mean_df['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
            mean_df['Strain'] = temp_df.loc[week_n:,'Load'].sum()*mean_df['Mono']
            mean_df['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[month_n:,'Load'].sum()


        team_df = team_df.append(mean_df, ignore_index=True)
        team_df.to_excel(writer, sheet_name='Team sheet', columns=team_df.columns, index=False, float_format='%.2f')


        for i in range(len(train_df)):
            player = train_df.iloc[i].Name
            position = train_df.iloc[i].Position
            sheet_name = player+'_'+position
            player_data = train_df.iloc[i:i+1,3:]
            player_df = pd.read_excel('./result_data/result_by_player.xlsx', sheet_name=sheet_name)

            temp_df = player_df.iloc[:,:-3]
            temp_df = temp_df.append(player_data, ignore_index=True)

            t_day = len(temp_df)

            if t_day <= 7:
                player_data['Mono'] = temp_df['Load'].mean()/temp_df['Load'].std()
                player_data['Strain'] = temp_df['Load'].sum()*player_data['Mono']
                player_data['EWAM'] = temp_df['Load'].sum()/temp_df['Load'].sum()
            elif t_day <=28:
                week_n = t_day - 7
                player_data['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
                player_data['Strain'] = temp_df.loc[week_n:,'Load'].sum()*player_data['Mono']
                player_data['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[week_n:,'Load'].sum()
            else:
                week_n = t_day - 7
                month_n = t_day - 28
                player_data['Mono'] = temp_df.loc[week_n:,'Load'].mean()/temp_df.loc[week_n:,'Load'].std()
                player_data['Strain'] = temp_df.loc[week_n:,'Load'].sum()*player_data['Mono']
                player_data['EWAM'] = temp_df.loc[week_n:,'Load'].sum()/temp_df.loc[month_n:,'Load'].sum()

            player_df = player_df.append(player_data, ignore_index=True)
            player_df.to_excel(writer, sheet_name=sheet_name, index=False , float_format='%.2f')

        writer.save()

        print("We have result file. Add data on result file")
        print("We workout {} times.".format(t_day))
        print("========================================================================================")

    else:
        writer = pd.ExcelWriter('./result_data/result_by_player.xlsx')
        mean_df['Mono'] = mean_df['Load'].mean()/mean_df['Load'].std()
        mean_df['Strain'] = mean_df['Load'].sum()*mean_df['Mono']
        mean_df['EWAM'] = mean_df['Load'].sum()/mean_df['Load'].sum()
        mean_df.to_excel(writer, sheet_name='Team sheet', columns=mean_df.columns, index=False, float_format='%.2f')

        for i in range(len(df)):
            player = df.iloc[i].Name
            position = df.iloc[i].Position
            sheet_name = player+'_'+position
            player_data = df.iloc[i:i+1,3:]
            player_data['Mono'] = player_data['Load'].mean()/player_data['Load'].std()
            player_data['Strain'] = player_data['Load'].sum()*player_data['Mono']
            player_data['EWAM'] = player_data['Load'].sum()/player_data['Load'].sum()
            player_data.to_excel(writer, sheet_name=sheet_name, columns=player_data.columns, index=False, float_format='%.2f')

        writer.save()

        print("Result file doesn't exist. Make result file")
        print("We workout 1 time")
        print("========================================================================================")

    print("Complete!!!!")

In [16]:
tr_data = 'WA_TR_20191015.xlsx'
td = trainig_merge(tr_data)

WA_TR_20191015.xlsx has 2 sheets
Now try to merge Training Dataset : Sheet1 at ./training_data/WA_TR_20191015.xlsx
Now try to merge Training Dataset : Sheet2 at ./training_data/WA_TR_20191015.xlsx
All Training Dataets were successfully merged.


In [17]:
td.head()

,No.,Name,Position,Date,Type,Height,Weight,RPE,TR_time_(min),Total_dist_(m),Dist_per_min_(m),HSR+Sprint_(m),MSR_(m),HSR_(m),Sprint_(m),Accel_(m),Decel_(m),Max_speed_(km/h),Load
0,1,Kim min jung,GK,20191015,필드 훈련 / Match,175,57.2,32.10,69.0,11675.8,85.9,413.8,1434.4,377.8,36.0,10,10,28.2,533.2
1,2,Kang ga ae,GK,20191015,필드 훈련 / Match,172,54.5,30.75,101.0,16126.2,81.4,522.4,1615.2,423.6,98.8,10,10,28.3,802.0
2,3,Min yu kyeong,GK,20191015,필드 훈련 / Match,176,60.7,33.85,102.0,17883.8,90.3,838.0,2076.4,655.6,182.4,10,10,30.2,875.1
3,4,Lim seon joo,CB,20191015,필드 훈련 / Match,168,59.3,33.65,101.0,18222.2,92.0,649.0,2067.8,569.2,79.8,10,10,28.6,910.6
4,5,Hong hye ji,CB,20191015,필드 훈련 / Match,174,60.9,32.95,102.0,18506.6,93.5,191.8,1633.4,191.8,0.0,10,10,24.7,677.7


In [18]:
result_by_player(td)

We have result file. Add data on result file
We workout 11 times.
Complete!!!!


In [8]:
for file in file_list:
    result_by_player(file)

Training dataset :  Sheet1 at ./player_data/WA_TR_20191015.xlsx
Result file doesn't exist. Make result file
We workout 1 time
Training dataset :  Sheet2 at ./player_data/WA_TR_20191015.xlsx
We have result file. Add data on result file
We workout 2 times.
Complete!!!!
Training dataset :  Sheet1 at ./player_data/WA_TR_20191016.xlsx
We have result file. Add data on result file
We workout 3 times.
Training dataset :  Sheet2 at ./player_data/WA_TR_20191016.xlsx
We have result file. Add data on result file
We workout 4 times.
Complete!!!!
Training dataset :  시트1 at ./player_data/WA_TR_20191018.xlsx
We have result file. Add data on result file
We workout 5 times.
Complete!!!!
Training dataset :  시트1 at ./player_data/WA_TR_20191019.xlsx
We have result file. Add data on result file
We workout 6 times.
Complete!!!!
Training dataset :  시트1 at ./player_data/WA_TR_20191020.xlsx
We have result file. Add data on result file
We workout 7 times.
Complete!!!!
Training dataset :  시트1 at ./player_data/WA_